In [ ]:
%pip install -r requirements.txt
%pip install pandas
%pip install plotly

In [9]:
import numpy as np
import pandas as pd
import torch


test_df = pd.read_csv('dataset/processed/test.csv')  
train_df = pd.read_csv('dataset/processed/training.csv')  
validation_df = pd.read_csv('dataset/processed/validation.csv') 
import plotly.graph_objects as go

torch.set_default_dtype(torch.float64)


In [10]:
trace1 = go.Scatter(
    x = train_df.timestamp,
    y = train_df.value,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = 'Signal Value Plot',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1], layout=layout)
fig.show()


In [12]:
from sklearn.preprocessing import StandardScaler


class AnomalyDetectionDataset(torch.utils.data.Dataset):

    def __init__(self, data, n_past=14, n_future=1):
        self.n_future = n_future
        self.n_past = n_past
        self.x = []
        self.actual = []

        self.scaler = StandardScaler()

        data_columns = ['value', 'predicted', "is_anomaly"]

        x = data[data_columns].astype(float)
        self.scaler = self.scaler.fit(x)
        x_scaled = self.scaler.transform(x)

        self.X = []
        self.Y = []
        self.IsAnomaly = []

        for i in range(self.n_past, len(x_scaled) - self.n_future + 1):
            self.X.append(  torch.tensor(x_scaled[i - self.n_past:i, 0: x_scaled.shape[1]])  )
            self.Y.append(  torch.tensor(x_scaled[i: i + self.n_future, 0 ])  )
            self.IsAnomaly.append(  torch.tensor(x_scaled[i:i + self.n_future, 2])  )

        assert len(self.X) == len(self.Y)

    def __len__(self):
        return len(self.X)


    def __getitem__(self, idx):
        return {
            "x": self.X[idx],
            "y": self.Y[idx],
            "is_anomaly": self.IsAnomaly[idx]
        }


def collate_batch(batch):
    x = []
    y = []
    is_anomaly = []

    for item in batch:
        x.append(item["x"])
        y.append(item["y"])
        is_anomaly.append(item["is_anomaly"])

    x = torch.nn.utils.rnn.pad_sequence(x, batch_first=True)
    y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True)
    is_anomaly = torch.nn.utils.rnn.pad_sequence(is_anomaly, batch_first=True)

    return {
        "x": x,
        "y": y,
        "is_anomaly": is_anomaly
    }

In [13]:
import torch.nn as nn
import torch


class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class EncoderDecoderConvLSTM(nn.Module):
    def __init__(self, nf, in_chan):
        super(EncoderDecoderConvLSTM, self).__init__()

        """ ARCHITECTURE 

        # Encoder (ConvLSTM)
        # Encoder Vector (final hidden state of encoder)
        # Decoder (ConvLSTM) - takes Encoder Vector as input
        # Decoder (3D CNN) - produces regression predictions for our model

        """
        self.encoder_1_convlstm = ConvLSTMCell(input_dim=in_chan,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.encoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_1_convlstm = ConvLSTMCell(input_dim=nf,  # nf + 1
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_CNN = nn.Conv3d(in_channels=nf,
                                     out_channels=1,
                                     kernel_size=(1, 3, 3),
                                     padding=(0, 1, 1))


    def autoencoder(self, x, seq_len, future_step, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4):

        outputs = []

        # encoder
        for t in range(seq_len):
            h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :],
                                               cur_state=[h_t, c_t])  # we could concat to provide skip conn here
            h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t,
                                                 cur_state=[h_t2, c_t2])  # we could concat to provide skip conn here

        # encoder_vector
        encoder_vector = h_t2

        # decoder
        for t in range(future_step):
            h_t3, c_t3 = self.decoder_1_convlstm(input_tensor=encoder_vector,
                                                 cur_state=[h_t3, c_t3])  # we could concat to provide skip conn here
            h_t4, c_t4 = self.decoder_2_convlstm(input_tensor=h_t3,
                                                 cur_state=[h_t4, c_t4])  # we could concat to provide skip conn here
            encoder_vector = h_t4
            outputs += [h_t4]  # predictions

        outputs = torch.stack(outputs, 1)
        outputs = outputs.permute(0, 2, 1, 3, 4)
        outputs = self.decoder_CNN(outputs)
        outputs = torch.nn.Sigmoid()(outputs)

        return outputs

    def forward(self, x, future_seq=0, hidden_state=None):

        """
        Parameters
        ----------
        input_tensor:
            5-D Tensor of shape (b, t, c, h, w)        #   batch, time, channel, height, width
        """

        # find size of different input dimensions
        b, seq_len, _, h, w = x.size()

        # initialize hidden states
        h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t3, c_t3 = self.decoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t4, c_t4 = self.decoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))

        # autoencoder forward
        outputs = self.autoencoder(x, seq_len, future_seq, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4)

        return outputs

In [15]:
from itertools import cycle
from tqdm import trange



batch_len = 1
input_channels = 1 # 1
output_channels = 1 # 1
height = 14 ## Number of previous datapoints to use
width = 3 ## Number of features per datapoints


plot_loss = []
plot_val_loss = []

train_dataset = AnomalyDetectionDataset(train_df, n_past=height, n_future=output_channels)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

# use the first half of the normal dev data as the dev set for the LSTM
split = np.array_split(validation_df, 2)
val_data = split[0]
val_dataset = AnomalyDetectionDataset(val_data, n_past=height,n_future=output_channels)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

epochs = 100
iterations = 200

loss_function = torch.nn.MSELoss(reduction='sum')

len_train = len(train_dataloader)
len_val = len(val_dataloader)


# instantiate model
autoEncoder = EncoderDecoderConvLSTM(nf=64, in_chan=1)
optimizer = torch.optim.Adam(autoEncoder.parameters(), lr=0.01)

train_iter = cycle(train_dataloader)
val_iter = cycle(val_dataloader)

for epoch in range(epochs):
    print("epoch", epoch)
    total_train_loss = 0
    total_val_loss = 0
    
    for itr in trange(len_train):

        autoEncoder.train()
        batch = next(train_iter, None)
        loss = 0
        
        input = torch.unsqueeze(batch['x'], 1)
        input = torch.unsqueeze(input, 1)
        pred = autoEncoder(input, output_channels).squeeze()

        loss = loss_function(pred[0][2], batch["is_anomaly"][0][0])

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        total_train_loss += loss.item()

    for itr in trange(len_val):
        autoEncoder.eval()
        batch = next(val_iter, None)
        loss=0
        input = torch.unsqueeze(batch['x'], 1)
        input = torch.unsqueeze(input, 1)
        pred = autoEncoder(input, output_channels).squeeze()
        loss = loss_function(pred[0][2], batch["is_anomaly"][0][0])
        total_val_loss += loss.item()

    total_train_loss /= (len_train * batch_len * height)
    total_val_loss /= (len_val * batch_len * height)

    print('epoch', epoch, 'loss', total_train_loss,'val_loss', total_val_loss)
    plot_loss.append(total_train_loss)
    plot_val_loss.append(total_val_loss)
    

epoch 0


 64%|██████▎   | 8035/12650 [02:58<01:53, 40.78it/s]